In [ ]:
import numpy as np
import os
import time

from lmfit.models import GaussianModel, Model
from sklearn.linear_model import LinearRegression
from sklearn import mixture, metrics

from shapely.geometry import Polygon

from scipy.stats.distributions import norm
from sklearn.neighbors import KernelDensity
from scipy.stats import gaussian_kde

from matplotlib import pyplot as plt
from matplotlib import cm

from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse

from scipy import ndimage as ndi
from scipy.spatial import distance
from numpy import linalg

import skimage
from skimage import io, transform
from skimage.draw import ellipse
from skimage.color import rgb2gray
from skimage import filters
from skimage.morphology import disk
from skimage import color
from skimage import filters, util
from skimage.morphology import disk, skeletonize, ball

from PIL import Image, ImageDraw, ImageFilter, ImageOps
import copy
import cv2
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.ndimage import distance_transform_edt as edt
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon
import logging
import glob
from radio_beam.commonbeam import getMinVolEllipse

from src.utils import grainPreprocess, grainShow, grainMark, grainDraw, grainApprox, grainStats, grainMorphology,grainGenerate
from src.cfg import CfgAnglesNames, CfgBeamsNames, CfgDataset


In [ ]:
def img_load_preprocess(path, img_names, h=150, k=1):
    imgs = []
    for i, class_img_names in enumerate(img_names):
        imgs.append([])
        for name in class_img_names:
            img = io.imread(path + '/' + name)
            img = grainPreprocess.image_preprocess(img, h, k)
            imgs[i].append(img)
    return imgs

In [ ]:
path = 'data/images'
all_images = np.load('data/saved np/all_images.npy', allow_pickle=True)

grain_names = np.array(
    [['Ultra_Co8.jpg'], ['Ultra_Co11.jpg'], ['Ultra_Co6_2.jpg'], ['Ultra_Co15.jpg'], ['Ultra_Co25.jpg']])
types = ['средние зерна', 'мелкие зерна', 'мелкие зерна', 'крупные зерна', 'средне-мелкие зерна']
grain_images = img_load_preprocess(path, grain_names)

### Считаем все изображения, предобработаем их и сохраним

In [ ]:
# images_dir='../data/dataset_4x/'
# images_dir='images/artif/'
# save_name='images_4x_(512,512,1).npy'
# save_name='artif.npy'

images_dir = '../data/dataset_sub_base_left/'
save_name = '(1500,1500,1)_sub_base_left'

all_images = grainPreprocess.read_preprocess_data(images_dir=images_dir, save_name=save_name,
                                                  max_images_num_per_class=90,
                                                  preprocess=True, save=True, crop_bottom=False, resize=True,
                                                  resize_shape=(1500, 1500, 1))

In [ ]:
image = io.imread('../data/dataset_sub_base_left/Ultra_Co8/Ultra_Co8-001.jpeg').astype(np.uint8)

## Распределение диаметров

In [ ]:
pixel = 49.7 / 1000
step = 5

# images = np.load('../data/saved np/images_4x_preprocess.npy')
# images = np.load('artif.npy')

# images = np.load('images_left_(1500,1500,1)_preprocessed.npy')
images = np.load('data/saved_np/images_10_mixer_preprocess.npy')
# names = np.load('metadata_left_(1500,1500,1)_preprocessed.npy')
names = np.load('data/saved_np/metadata_10_mixer_preprocess.npy')
names = [name.split('\\')[-1] for name in names]

folder = 'data/'

# path = 'data/saved np'
path = 'data/saved_np/'
# name = 'artif'
# name = 'left_(1500,1500,1)'
name = '10_mixer_preprocess'

# types_dict = {'Ultra_Co11': 'средние зерна',
#               'Ultra_Co25': 'мелкие зерна',
#               'Ultra_Co8': 'средне-мелкие зерна',
#               'Ultra_Co6_2': 'крупные зерна',
#               'Ultra_Co15': 'средне-мелкие зерна'}

types_dict = {'class_1': 'Type None'}

for step in range(1, 6):
    # print('step: ',step)
    grainGenerate.diametr_approx_save(folder, images,name, names, types_dict, step, pixel, save=True)

In [ ]:
def diametr_approx_all(folder,name, step, N, M, indeces=None,save=False):
    #
    # вывод распределения длин а- и б- полуосей для разных образцов
    #

    xy_scatter = np.load(f'{folder}/' + CfgBeamsNames.values + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear = np.load(f'{folder}/' + CfgBeamsNames.approx + f'{name}' + f'{step}.npy', allow_pickle=True)
    xy_linear_data = np.load(f'{folder}/' + CfgBeamsNames.approx_data + f'{name}' + f'{step}.npy', allow_pickle=True)
    texts = np.load(f'{folder}/' + CfgBeamsNames.legend + f'{name}' + f'{step}.npy', allow_pickle=True)

    if indeces is not None:
        xy_scatter = xy_scatter[indeces]
        xy_linear = xy_linear[indeces]
        xy_linear_data = xy_linear_data[indeces]
        texts = texts[indeces]

    plt.rcParams['font.size'] = '20'
    gs = gridspec.GridSpec(2, 1)
    # для статьи 12,18
    plt.figure(figsize=(N, M))

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])

    ax1.set_title('распределение a-полуосей связующего вещества', fontsize=20)
    ax2.set_title('распределение b-полуосей связующего вещества', fontsize=20)

    ax1.set_ylabel('ln(p(x))', fontsize=15)
    ax1.set_xlabel('длина полуоси, мкм', fontsize=15)

    ax2.set_ylabel('ln(p(x))', fontsize=15)
    ax2.set_xlabel('длина полуоси, мкм', fontsize=15)

    #  ax3.grid()
    markers = ['v', 's', 'D', 'd', 'p', '*']
    for i in range(len(xy_scatter)):
        ax1.plot(xy_linear[i][0][0], xy_linear[i][0][1])
        ax2.plot(xy_linear[i][1][0], xy_linear[i][1][1])

    ax1.legend(texts[:, 0], fontsize=14, loc=1)
    ax2.legend(texts[:, 1], fontsize=14, loc=1)

    for i in range(len(xy_scatter)):
        marker = markers[i]
        ax1.scatter(xy_scatter[i][0][0], xy_scatter[i][0][1], marker=marker)
        ax2.scatter(xy_scatter[i][1][0], xy_scatter[i][1][1], marker=marker)

    if save:
        plt.savefig(f'распределение_углов_{name}_шаг_{step}')

    plt.show()


In [ ]:
N = 30
M = 30
# для статье N=12,M=18

step = 1
save=True

# folder = '../data/saved np'
folder = 'data/saved_np'
name = '10_mixer_preprocess'
# name = ''



# diametr_approx_all(folder,name, step, N, M, indeces=[0, 2, 3])
diametr_approx_all(folder,name, step, N, M, indeces=None,save=save)

### Вывод на снимке ненормалных углов

In [ ]:
def draw_edges(image, tol=3, border_eps=5):
    #
    # рисует на изображении линии по точкам контура cnts
    # линии в стиле x^1->x^2,x^2->x^3 и тд
    # на вход подавать обработанное изображение
    #
    new_image = image[:, :, 0]
    im = Image.fromarray(np.uint8(new_image)).convert('RGB')
    draw = ImageDraw.Draw(im)

    cnts = grainMark.get_row_contours(image)
    angles = []

    for j, cnt in enumerate(cnts):
        rules = [image.shape[0] - cnt[:, 0].max() > border_eps, cnt[:, 0].min() > border_eps,
                 image.shape[1] - cnt[:, 1].max() > border_eps, cnt[:, 1].min() > border_eps]
        # проверяем находится ли контур у границы, но это срабатывает очень редко
        if all(rules):
            cnt_aprox = approximate_polygon(cnt, tolerance=tol)
            # основная проверка на неправильные контуры
            if len(cnt_aprox) > 3:
                for i, point in enumerate(cnt_aprox[:-1]):

                    y1, x1 = cnt_aprox[i - 1]
                    y2, x2 = cnt_aprox[i]
                    y3, x3 = cnt_aprox[i + 1]

                    v1 = np.array((x1 - x2, y1 - y2)).reshape(1, 2)
                    v2 = np.array((x3 - x2, y3 - y2)).reshape(1, 2)

                    dot = np.dot(v1[0], v2[0])
                    dist1 = np.linalg.norm(v1[0])
                    dist2 = np.linalg.norm(v2[0])
                    cos = dot / (dist1 * dist2)

                    v = np.concatenate([v1, v2])
                    det = np.linalg.det(v)

                    if abs(cos) < 1:
                        ang = int(np.arccos(cos) * 180 / np.pi)

                        if det < 0:
                            angles.append(ang)
                        else:
                            angles.append(360 - ang)
                    else:
                        if det < 0:
                            angles.append(360)
                            ang = 360
                        else:
                            angles.append(0)
                            ang = 0

                    line_color = (0, 255, 127)

                    if ang <= 30:
                        color = (0, 0, 255)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        draw.line((y1, x1, y2, x2), fill=line_color, width=4)
                        draw.line((y3, x3, y2, x2), fill=line_color, width=4)

                    elif ang >= 30:
                        color = (255, 0, 0)
                        r = 4
                        draw.ellipse((y2 - r, x2 - r, y2 + r, x2 + r), fill=color, width=5)
                        # draw.line((y1, x1, y2, x2), fill=line_color, width=4)
                        # draw.line((y3, x3, y2, x2), fill=line_color, width=4)


        else:
            continue

    img = np.array(im)

    return img, angles

In [ ]:
# img=io.imread('../data/dataset_4x/Ultra_Co8/Ultra_Co8-001_1.jpeg')
i = 0
img = np.load('data/artif.npy')[1, i]
tol = 3
border_eps = 5
im, angles = draw_edges(img, tol, border_eps)
io.imsave(f'low_angles_artif.png', im)
grainShow.img_show(im, 10, cmap='gray')
